In [2]:
COMPETITION_NAME = 'tabular-playground-series-nov-2022' # replace me

# Kaggle competition bootstrapping
!pip install -qq fastai
!pip install -qq kaggle

from fastai.imports import *
np.set_printoptions(linewidth=130)

# set up kaggle token
import json 
import os
from pathlib import Path

# get competition data
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if not iskaggle:
    api_key = {"username":"dogxyz","key":"f39bca0d1d833d63fa7dad15450de268"}

    kaggle_path = Path('/root/.kaggle') 
    # uncomment if running locally on wsl
    # kaggle_path = Path('/home/ubuntu/.kaggle') 

    os.makedirs(kaggle_path, exist_ok=True)

    with open (kaggle_path/'kaggle.json', 'w') as handl:
        json.dump(api_key,handl)

    !chmod 600 /root/.kaggle/kaggle.json


if iskaggle: path = Path('../input/'+COMPETITION_NAME)
else:
    import zipfile,kaggle
    path = Path(COMPETITION_NAME)
    if not path.is_dir():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)
print('done')

tabular-playground-series-nov-2022.zip: Skipping, found more recently modified local copy (use --force to force download)
done


## Preprocess and clean up data

In [3]:
!ls {path}

sample_submission.csv  submission_files  train_labels.csv


In [4]:
!ls {path}/submission_files | head

0.6222863195.csv
0.6223807245.csv
0.6225426578.csv
0.6247722291.csv
0.6253455681.csv
0.6254850917.csv
0.6255093621.csv
0.6260141578.csv
0.6263493693.csv
0.6272779211.csv
ls: write error: Broken pipe


In [5]:
labels = pd.read_csv(path/'train_labels.csv', index_col='id')
labels.head()

,label
id,
0,0
1,1
2,1
3,1
4,0


In [6]:
subs = sorted(os.listdir(path / 'submission_files'))
subs[:5]

['0.6222863195.csv',
 '0.6223807245.csv',
 '0.6225426578.csv',
 '0.6247722291.csv',
 '0.6253455681.csv']

In [7]:
s0 = pd.read_csv(path/'submission_files'/subs[0])
print(subs[0])
print(len(subs))
s0

0.6222863195.csv
5000


,id,pred
0,0,0.709336
1,1,0.452988
2,2,0.675462
3,3,0.481046
4,4,0.957339
...,...,...
39995,39995,0.382515
39996,39996,0.352498
39997,39997,0.577554
39998,39998,0.712353


In [8]:
sn =  pd.read_csv(path/'submission_files'/subs[-1])
sn

,id,pred
0,0,0.787
1,1,0.547
2,2,0.667
3,3,0.622
4,4,0.934
...,...,...
39995,39995,0.266
39996,39996,0.725
39997,39997,0.572
39998,39998,0.641


## Prepare submission file

In [9]:
submission = pd.read_csv(path/'sample_submission.csv', index_col='id')
submission

,pred
id,
20000,0.640707
20001,0.636904
20002,0.392496
20003,0.588658
20004,0.783603
...,...
39995,0.382515
39996,0.352498
39997,0.577554


In [10]:
submission.shape

(20000, 1)

## Let's try understanding this log_loss thing

In [11]:
labels

,label
id,
0,0
1,1
2,1
3,1
4,0
...,...
19995,1
19996,1
19997,0


In [12]:
from sklearn.metrics import log_loss


sub_ids = submission.index
gt_ids = labels.index

s0 = pd.read_csv(path / 'submission_files' / subs[0], index_col='id')

score = log_loss(labels, s0.loc[gt_ids])

# Notice the score of the labeled rows matches the file name
print(subs[0], f'{score:.10f}')

0.6222863195.csv 0.6222863195


In [13]:
s1 = pd.read_csv(path / 'submission_files' / subs[1], index_col='id')
score = log_loss(labels, s1.loc[gt_ids])
print(subs[1], f'{score:.10f}')

sn = pd.read_csv(path / 'submission_files' / subs[-1], index_col='id')
score = log_loss(labels, sn.loc[gt_ids])
print(subs[-1], f'{score:.10f}')

0.6223807245.csv 0.6223807245
0.7575039918.csv 0.7575039918


In [38]:
from tqdm.notebook import tqdm
import random
epochs = 20

overall = s0
best_overall = log_loss(labels, s0.loc[gt_ids])

for epoch in tqdm(range(epochs)):
    blend = s0
    best_score = log_loss(labels, s0.loc[gt_ids])

    sub_idxs = list(range(len(subs)))
    #random.shuffle(sub_idxs)
    for i in (sub_idxs):
        si = pd.read_csv(path / 'submission_files' / subs[i], index_col='id')
        # 0.5805080563816475
        #potential = (blend + si) / 2
        
        r = random.random()
        potential = (r*blend + si*(1-r)) / 1
        score = log_loss(labels, potential.loc[gt_ids])
        #print('potential', f'{score:.10f}')
        if score < best_score:
            #print('new best!', score)
            best_score = score
            blend = potential
    if best_score < best_overall:
        print('new overall best!!!', best_score)
        best_overall = best_score
        overall = blend

  0%|          | 0/20 [00:00<?, ?it/s]

new overall best!!! 0.5832083217170534
new overall best!!! 0.5811998374744681
new overall best!!! 0.5752190778626524
new overall best!!! 0.5706016673791618


In [39]:
score = log_loss(labels, overall.loc[gt_ids])
score

0.5706016673791618

In [40]:
submission = overall.loc[sub_ids]
submission['id'] = submission.index
submission = submission[['id', 'pred']]
submission

,id,pred
id,,
20000,20000,0.618407
20001,20001,0.722784
20002,20002,0.352882
20003,20003,0.488477
20004,20004,0.326931
...,...,...
39995,39995,0.321069
39996,39996,0.450805
39997,39997,0.578362


## Submit CSV to Kaggle

In [35]:
import time
for sec in tqdm(range(9000)):
    time.sleep(1)

  0%|          | 0/9000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [41]:
# write submission csv
technique = 'best of 20 random blends in random order'
sub_filename = Path('subs')/(COMPETITION_NAME+'-'+technique+'-sub.csv')
submission.to_csv(sub_filename, index=False)

if not iskaggle:
    #for sec in tqdm(range(9000)):
    #    time.sleep(1)
    print('about to submit', sub_filename, 'to kaggle')
    ret = kaggle.api.competition_submit(sub_filename, technique, COMPETITION_NAME)
    print(ret)

about to submit subs/tabular-playground-series-nov-2022-best of 20 random blends in random order-sub.csv to kaggle


100%|██████████| 490k/490k [00:00<00:00, 1.10MB/s]


Successfully submitted to Tabular Playground Series - Nov 2022
